# Getting Started with EEG Data

In 2011-2012, the brain-computer interface (BCI) research group at Colorado State University recorded EEG signals from subjects in our lab and in their homes, using three different EEG systems. One goal of this work is to determine if inexpensive EEG systems (about $7,000) are as effective as more expensive ones (about $40,000) for conducting BCI experiments in the home.

On this page, we summarize the steps you can follow to download some of the data, load it into an ipython environment, and visualize it. We also show examples of looking at P300 ERP’s.

# Downloading EEG Data

EEG data from multiple subjects can be downloaded from our Public BCI Data site. Let’s select the data files for the first subject in each device column, for subjects recorded in our lab.

![Data download page](http://www.cs.colostate.edu/eeg/data/json/doc/tutorial/_build/html/_images/eegDownload.png)

# Extract the downloaded data
The zip file should contain six zipped data files. Extract the files, for example using the following commands:

```
> cd ~/Download

> unzip eeg.zip
Archive:  eeg.zip
 extracting: s20-activetwo-gifford-unimpaired.json.zip
 extracting: s21-activetwo-gifford-unimpaired.json.zip
 extracting: s20-gammasys-gifford-unimpaired.json.zip
 extracting: s21-gammasys-gifford-unimpaired.json.zip
 extracting: s20-mindset-gifford-unimpaired.json.zip
 extracting: s21-mindset-gifford-unimpaired.json.zip

> rm eeg.zip

> ls -l --block-size=M *json*
-rw-r--r-- 1 ... 84M Mar 12 10:50 s20-activetwo-gifford-unimpaired.json.zip
-rw-r--r-- 1 ...  5M Mar 12 10:50 s20-gammasys-gifford-unimpaired.json.zip
-rw-r--r-- 1 ... 29M Mar 12 10:50 s20-mindset-gifford-unimpaired.json.zip
-rw-r--r-- 1 ... 80M Mar 12 10:51 s21-activetwo-gifford-unimpaired.json.zip
-rw-r--r-- 1 ...  5M Mar 12 10:51 s21-gammasys-gifford-unimpaired.json.zip
-rw-r--r-- 1 ... 28M Mar 12 10:52 s21-mindset-gifford-unimpaired.json.zip

> unzip s20-gammasys-gifford-unimpaired.json.zip
Archive:  s20-gammasys-gifford-unimpaired.json.zip
  inflating: s20-gammasys-gifford-unimpaired.json

> unzip s20-mindset-gifford-unimpaired.json.zip
Archive:  s20-mindset-gifford-unimpaired.json.zip
  inflating: s20-mindset-gifford-unimpaired.json

> unzip s20-activetwo-gifford-unimpaired.json.zip
Archive:  s20-activetwo-gifford-unimpaired.json.zip
  inflating: s20-activetwo-gifford-unimpaired.json

> rm s20*zip
```

The unzipped data can loaded into an ipython environment.

In [90]:
# Show matplotlib plots in this notebook
%matplotlib inline

# Imports
import matplotlib.pyplot as plt
import numpy as np
import json
import scipy.signal as sig

In [91]:
# Open the data file
gammasys_data_file = open('data/s20-gammasys-gifford-unimpaired.json','r')

# Parse data to JSON
data = json.load(gammasys_data_file)

The variable data is a list of dictionaries, each with the same keys.

In [92]:
len(data)

8

In [93]:
data[0].keys()

dict_keys(['protocol', 'device', 'sample rate', 'channels', 'impairment', 'date', 'notes', 'location', 'subject', 'eeg'])

Here is a handy function to show keys and their values in each data element.

In [94]:
def summarize(datalist):
    for i,element in enumerate(datalist):
        keys = list(element.copy().keys())
        print('\nData set', i)
        keys.remove('eeg')
        for key in keys:
            print('  {}: {}'.format(key,element[key]))
        eegtrials = element['eeg']
        shape = np.array(eegtrials['trial 1']).shape
        print ('  eeg: {:d} trials, each a matrix with {:d} rows and approximately {:d} columns').format(  len(eegtrials), shape[0], shape[1])

In [95]:
summarize(data)


Data set 0
  device: GAMMAsys
  date: [2012, 3, 8]
  location: gifford
  notes: 
  protocol: 3minutes
  subject: 20
  sample rate: 256
  impairment: none
  channels: ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
  eeg: {:d} trials, each a matrix with {:d} rows and approximately {:d} columns


AttributeError: 'NoneType' object has no attribute 'format'

# Plotting some EEG

The first element of the data list has key-value pair `protocol: 3minutes`, meaning that this element contains 3 minutes of EEG recorded while the subject was asked to relax and look at the computer screen. Let’s take a look at 2 seconds of this data.

The EEG consists of one matrix with 9 rows and 46,342 columns. The 9 rows correspond to the channels `channels:  ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']` plus one more channel that is used to mark stimuli onset and offset, which is not used for the 3 minute protocol. The number of samples (in columns) in one second depends on the sample rate, which for this device, `device: GAMMAsys`, is 256 samples per second, `sample rate: 256`. Let’s plot data from all 9 channels for columns 1,000 to 1,512.

In [ ]:
first = data[0]
eeg = np.array(first['eeg']['trial 1'])
eeg.shape

In [ ]:
# Using ending semicolon to suppress output of plotting functions.
plt.figure(1);

plt.axis('tight');

plt.plot(eeg[:,4000:4512].T);

Kind of hard to see each channel. Let’s spread them out and not plot the constant, unused, 9th channel. Also, we can add a legend with the channel names. If we reverse the vertical order of the channel plots, they will correspond with the vertical order of the channel names.

In [ ]:
plt.figure(2);

plt.plot(eeg[:8,4000:4512].T + 80*np.arange(7,-1,-1));

plt.plot(np.zeros((512,8)) + 80*np.arange(7,-1,-1),'--',color='gray');

plt.yticks([]);

plt.legend(first['channels']);

plt.axis('tight');

# Again, for EEG from ActiveTwo and Mindset Systems¶

Now let’s summarize the data from the other two systems. First, rename data to dataGammasys.

In [ ]:
dataGammasys = data

dataActivetwo = json.load(open('data/s20-activetwo-gifford-unimpaired.json','r'))

dataMindset = json.load(open('data/s20-mindset-gifford-unimpaired.json','r'))

summarize(dataMindset[0:2])

This shows that the Mindset has 19 channels of EEG, but the EEG matrix has 24 rows. The first 19 rows are the EEG channels. Let’s plot them.

In [ ]:
eegMindset = np.array(dataMindset[0]['eeg']['trial 1'])

plt.figure();

plt.plot(eegMindset[:19,4000:4512].T + 30*np.arange(18,-1,-1));

plt.plot(np.zeros((512,19)) + 30*np.arange(18,-1,-1),'--',color='gray');

plt.yticks([]);

plt.legend(dataMindset[0]['channels'], prop={'size':10});

plt.axis('tight');

Now for the data from the ActiveTwo system. First, let’s see which element in the list is for the 3minutes protocol.

In [ ]:
summarize(dataActivetwo[0:2])

In [ ]:
eegActivetwo = np.array(dataActivetwo[1]['eeg']['trial 1'])

eegActivetwo.shape

This data matrix contains 41 rows. The list of channels is the 41 names

In [ ]:
dataActivetwo[1]['channels']

The channels named EXG1 through EXG6 contain non-EEG data as follows:

| Channel | Index | Electrode |
|--- | --- | --- |
| EXG1 | 32 | EOG vertical left |
| EXG2 | 33| EOG vertical right |
| EXG3 | 34| EOG horizontal left |
| EXG4 | 35| EOG horizontal right |
| EXG5 | 36| earlobe left |
| EXG6 | 37| earlobe right |

Typically, the EEG channels (indices 0 through 31) are referenced to the earlobes, after removing the linear trend. That’s easy.

In [ ]:
eegActivetwo = sig.detrend(eegActivetwo,1)

ref = np.mean(eegActivetwo[36:38,:],axis=0).reshape((1,-1))

eeg = eegActivetwo[:32,:] - ref

Now we can plot all 32 EEG channels.

In [ ]:
plt.figure();

plt.plot(eeg[:,4000:4512].T + 150*np.arange(31,-1,-1))

plt.plot(np.zeros((512,32)) + 150*np.arange(31,-1,-1),'--',color='gray');

plt.yticks([]);

plt.legend(dataActivetwo[0]['channels'][:32], prop={'size':8});

plt.axis('tight');